# 提出用notebook
* 2019/07/18, 19
* Best test accuracy: 0.962

## 目次
1. 概要
1. 自作パイプライン、便利関数
1. ネットワークモデル学習部分
1. TTA

## 1. 概要
* Augumentaion: ランダムクロップ（アス比固定）, ランダムフリップ(水平方向)
* モデル: VGG16の全結合層のファインチューン(VGG16部分は重み固定)
* Optimizer: Adam（Lrデフォルト）
* バッチサイズ: 32
* エポック数: 2
* 後処理： TTA(Test time augumentation）を実施し、最終予測値を出力

## 2. 自作パイプライン、便利関数

In [1]:
import numpy as np
import pandas as pd
import random
import json
import os
from glob import glob
from datetime import datetime

import chainer
import chainer.links as L
import chainer.functions as F
from chainer.datasets import LabeledImageDataset
from chainer.datasets import TransformDataset
from chainercv.transforms import resize, random_flip, random_crop, random_sized_crop, random_flip
from chainer.datasets import TupleDataset, split_dataset_random
from chainer.iterators import SerialIterator
from chainer.optimizers import Adam
from chainer.training import StandardUpdater, Trainer
from chainer.training.extensions import Evaluator, LogReport, PrintReport
from chainer.cuda import to_cpu

from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
# シードの固定関数、データ読み込み関数
def reset_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    if chainer.cuda.available:
        chainer.cuda.cupy.random.seed(seed)

def load_new_dataset(data_dir):
    train_dogs = glob(os.path.join(data_dir, 'train/dog/*.jpg'))
    train_cats = glob(os.path.join(data_dir, 'train/cat/*.jpg'))
    valid_dogs = glob(os.path.join(data_dir, 'validation/dog/*.jpg'))
    valid_cats = glob(os.path.join(data_dir, 'validation/cat/*.jpg'))
    test_dogs = glob(os.path.join(data_dir, 'test_v2/dog/*.jpg'))
    test_cats = glob(os.path.join(data_dir, 'test_v2/cat/*.jpg'))

    # 教師ラベルの作成
    train_label = {
        'cat': 1,
        'dog': 0}

    df = pd.DataFrame({
        'file_path': train_cats + train_dogs + valid_dogs + valid_cats + test_dogs + test_cats,
    })
    df['label'] = df['file_path'].str.split('/', expand=True)[5]
    df['dataset'] = df['file_path'].str.split('/', expand=True)[4]
    df['target'] = df['label'].replace(train_label)
    
    # データセットを作成
    train_df = df[df['dataset'] == 'train']
    valid_df = df[df['dataset'] == 'validation']
    test_df = df[df['dataset'] == 'test_v2']

    train = TupleDataset(train_df['file_path'].values, train_df['target'].values.astype('int32'))
    valid = TupleDataset(valid_df['file_path'].values, valid_df['target'].values.astype('int32'))
    test = TupleDataset(test_df['file_path'].values, test_df['target'].values.astype('int32'))

    return train, valid, test

In [3]:
# train, validに対して、前処理（＋オーグメンテーション）を実行し、ネットワークを学習させるクラス
class ChainerPipeline:
    def __init__(self, preprocess, network, train, valid, setting):
        self.preprocess = preprocess
        self.network = network
        
        self.train = train
        self.valid = valid
        self.setting = setting

    def run(self):
        # 前処理
        train = self.preprocess.transform(self.train)
        valid = self.preprocess.transform(self.valid)

        # モデル学習・評価
        model = self.chainer_model_pipe(self.network, train, valid, self.setting)

        # 結果可視化
        result = self.visualize_result(self.preprocess, self.network, self.setting)
        return model, result
    
    # chainerモデルのパイプライン
    def chainer_model_pipe(self, nn, train, valid, params):
        epoch = params['epoch']
        batch_size = params['batch_size']
        use_gpu = params['use_gpu']

        if 'fixed_base_w' in params.keys():
            fixed_base_w = params['fixed_base_w']
        else:
            fixed_base_w = False

        # Model Instance
        model = L.Classifier(nn)

        if use_gpu:
            device = 0
            model.to_gpu(device)
        else:
            device = -1

        # ミニバッチのインスタンスを作成
        train_iter = SerialIterator(train, batch_size)
        valid_iter = SerialIterator(valid, batch_size, repeat=False, shuffle=False)

        # Set Lerning
        optimizer = Adam()
        optimizer.setup(model)

        if fixed_base_w:
            model.predictor.base.disable_update()

        updater = StandardUpdater(train_iter, optimizer, device=device)

        trainer = Trainer(updater, (epoch, 'epoch'), out='result/cat_dog')
        trainer.extend(Evaluator(valid_iter, model, device=device))
        trainer.extend(LogReport(trigger=(1, 'epoch')))
        trainer.extend(PrintReport(['epoch', 'main/accuracy', 'validation/main/accuracy', 
                                    'main/loss', 'validation/main/loss', 'elapsed_time']), 
                       trigger=(1, 'epoch'))

        trainer.run()

        if use_gpu:
            model.to_cpu()

        return model
    
    # 結果の可視化
    def visualize_result(self, preprocess, network, setting, plot_learn=False, path='./result/cat_dog/'):
        with open(os.path.join(path, 'log')) as f:
            result = pd.DataFrame(json.load(f))

        log = pd.Series()
        log['Preprocess'] = preprocess.__class__.__name__
        log['Model'] = network.__class__.__name__
        log['Elapsed time'] = result.iloc[-1]['elapsed_time']
        log['Validation accuracy'] = result.iloc[-1]['validation/main/accuracy']
        log = log.append(pd.Series(setting))
        print(log)
        log.to_json(os.path.join(path, 'run', '%s.json' % datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
                  
        if plot_learn:
            result[['main/accuracy', 'validation/main/accuracy']].plot()
        return result

In [4]:
# 前処理、モデルのクラス
class Processing_11:
    def __init__(self):
        pass
    
    def transform(self, x):
        dataset = LabeledImageDataset(x)

        def _transform(in_data):
            img, label = in_data
            img = random_sized_crop(img, scale_ratio_range=(0.3, 1))
            img = random_flip(img, x_random=True)
            img = chainer.links.model.vision.vgg.prepare(img)
            return img, label
        
        return TransformDataset(dataset, _transform)
    
    
class VGG(chainer.Chain):
    def __init__(self, n_out=2):
        super().__init__()

        with self.init_scope():
            self.base = L.VGG16Layers()
            self.fc8 = L.Linear(None, n_out)

    def __call__(self, x):
        h = self.base(x, layers=['fc7'])['fc7']
        h = self.fc8(h)
        return h

In [5]:
# 学習・内部評価を実行する関数
def learn_network_model(preprocess_inst, network_inst, setting):
    reset_seed(0)
    train, valid, test = load_new_dataset('../new_dataset/dataset/data/')
       
    p = ChainerPipeline(preprocess_inst, network_inst, train, valid, setting)
    model, result = p.run()
    return model

In [6]:
# testの予測・アンサンブル用の関数
def get_target_label(tuple_dataset):
    return [tpl[1] for tpl in tuple_dataset]

def model_predict(tuple_dataset, model, gpu_id=0):
    model.to_gpu(gpu_id)

    predicted = []
    for img, label in tuple_dataset:
        img = np.array([img])
        img = model.xp.asarray(img)

        with chainer.using_config('train', False), chainer.using_config('enable_backprp', False):
            predict = model.predictor(img)

        predict = to_cpu(predict.data)
        predicted.append(np.argmax(predict))

    model.to_cpu()
    return predicted

def evaluate_predict(target_labels, predicted):
    print('Test data accuracy:', accuracy_score(test_t, predicted))
    print('Confusion_matrix:\n', confusion_matrix(test_t, predicted))

## 3. ネットワークの学習

モデル・前処理インスタンス・パラメータを入力して実行

In [7]:
setting = {
    'epoch': 2,
    'batch_size': 32,
    'use_gpu': True,
    'fixed_base_w': True
}

vgg1 = learn_network_model(Processing_11(), VGG(), setting)

epoch       main/accuracy  validation/main/accuracy  main/loss   validation/main/loss  elapsed_time
1           0.936508       0.9625                    0.192835    0.0926208             150.96        
2           0.96371        0.97                      0.108717    0.0894009             299.67        
Preprocess             Processing_11
Model                            VGG
Elapsed time                  299.67
Validation accuracy             0.97
batch_size                        32
epoch                              2
fixed_base_w                    True
use_gpu                         True
dtype: object


## 後処理

* TTA(Test Time augumentaion)を実施したものが一番良かった。
* https://openreview.net/forum?id=rJZz-knjz
* 複数のモデルを使って2段階の学習を行うStackingも実施したが、0.94程度だった

In [8]:
# load dataset: tuple(path, label)
train, valid, test = load_new_dataset('../new_dataset/dataset/data/')
len(train), len(valid), len(test)

(2000, 800, 1000)

In [9]:
# Data loader?を定義
preprocess = Processing_11()
train = preprocess.transform(train)
valid = preprocess.transform(valid)
test = preprocess.transform(test)

train_t = get_target_label(train)
valid_t = get_target_label(valid)
test_t = get_target_label(test)

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


In [10]:
# 素のモデルのテストデータ精度（※オーグメンテーションされたtestデータ）
predicted = model_predict(test, vgg1)
evaluate_predict(test_t, predicted)

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


Test data accuracy: 0.945
Confusion_matrix:
 [[457  43]
 [ 12 488]]


In [12]:
# TTAを実施
N = 20

predict_sum = np.zeros(len(test_t))
for i in range(N):
    predict_sum += model_predict(test, vgg1)
    if i%5 == 0:
        print('%s/%s complete' % (i+1, N))

predict_tta = (predict_sum/N > 0.5).astype(int)
evaluate_predict(test_t, predict_tta)

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:739: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


1/20 complete
6/20 complete
11/20 complete
16/20 complete
Test data accuracy: 0.962
Confusion_matrix:
 [[471  29]
 [  9 491]]
